In [9]:
import numpy as np
import pandas as pd
import music21

np.random.seed(401)

In [10]:
data = pd.read_csv('train.csv')
data.head()

,piece,beats,pitches
0,../abc/_Polonäs_ur_Andreas_Dahlgrens_notbok_nr...,"[[1, 3, 5, 7, 9, 11], [1, 3, 5, 9], [1, 5, 7, ...","[[70, 69, 67, 69, 70, 72], [74, 76, 77, 74], [..."
1,../abc/_Polska_efter_ByssKalle_nr_36_a115df.abc,"[[1, 3, 4, 5, 7, 8, 9, 11, 12], [1, 2, 3, 4, 5...","[[76, 76, 79, 77, 77, 79, 76, 76, 79], [77, 76..."
2,../abc/Polonäs_i_G_efter_Johan_Eric_Blomgren_4...,"[[1, 3, 5, 7, 9, 11, 12], [1, 2, 3, 4, 5, 6, 7...","[[67, 71, 71, 74, 74, 79, 74], [72, 71, 72, 76..."
3,../abc/Polska_från_Vrigstad_c46508.abc,"[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [1, ...","[[74, 73, 74, 73, 74, 73, 74, 73, 74, 73, 74, ..."
4,../abc/Polonäs_32c37e.abc,"[[1, 3, 5, 7, 9, 11], [1, 3, 5, 7, 9, 11], [1,...","[[62, 66, 69, 74, 78, 81], [81, 78, 79, 76, 78..."


In [11]:
import ast
# change the note column to list
data['beats'] = data['beats'].apply(ast.literal_eval)
data['pitches'] = data['pitches'].apply(ast.literal_eval)